In [4]:
# importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt

In [5]:
df=pd.read_csv(r'C:\Users\HP\OneDrive\Desktop\Uber Request Data.csv')

In [6]:
df.head(10)

,Request id,Pickup point,Driver id,Status,Request timestamp,Drop timestamp
0,619,Airport,1.0,Trip Completed,11/7/2016 11:51,11/7/2016 13:00
1,867,Airport,1.0,Trip Completed,11/7/2016 17:57,11/7/2016 18:47
2,1807,City,1.0,Trip Completed,12/7/2016 9:17,12/7/2016 9:58
3,2532,Airport,1.0,Trip Completed,12/7/2016 21:08,12/7/2016 22:03
4,3112,City,1.0,Trip Completed,13-07-2016 08:33:16,13-07-2016 09:25:47
5,3879,Airport,1.0,Trip Completed,13-07-2016 21:57:28,13-07-2016 22:28:59
6,4270,Airport,1.0,Trip Completed,14-07-2016 06:15:32,14-07-2016 07:13:15
7,5510,Airport,1.0,Trip Completed,15-07-2016 05:11:52,15-07-2016 06:07:52
8,6248,City,1.0,Trip Completed,15-07-2016 17:57:27,15-07-2016 18:50:51
9,267,City,2.0,Trip Completed,11/7/2016 6:46,11/7/2016 7:25


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6745 entries, 0 to 6744
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Request id         6745 non-null   int64  
 1   Pickup point       6745 non-null   object 
 2   Driver id          4095 non-null   float64
 3   Status             6745 non-null   object 
 4   Request timestamp  6745 non-null   object 
 5   Drop timestamp     2831 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 316.3+ KB


In [8]:
#  Check for null values and count the number of nulls for each column
null_counts = df.isnull().sum()

# Filter and display columns with more than 0 null values
columns_with_nulls = null_counts[null_counts > 0]
print(columns_with_nulls)

Driver id         2650
Drop timestamp    3914
dtype: int64


In [10]:
# checking the status to understand nulls in (Drop timestamp) column
df['Status'].value_counts()

Trip Completed       2831
No Cars Available    2650
Cancelled            1264
Name: Status, dtype: int64

## nulls in drop timestamp column due to cancelled and no cars available 

In [11]:
#Convert Request timestamp & Drop timestamep to uniform datetime format
df['Request timestamp'] = pd.to_datetime(df['Request timestamp'])
df['Drop timestamp'] = pd.to_datetime(df['Drop timestamp'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6745 entries, 0 to 6744
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Request id         6745 non-null   int64         
 1   Pickup point       6745 non-null   object        
 2   Driver id          4095 non-null   float64       
 3   Status             6745 non-null   object        
 4   Request timestamp  6745 non-null   datetime64[ns]
 5   Drop timestamp     2831 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1), object(2)
memory usage: 316.3+ KB


In [28]:
# Calculate trip duration in minutes
df['Trip duration'] = (df['Drop timestamp'] - df['Request timestamp']).dt.total_seconds() / 60

In [29]:
#  checking the new column
df.head()

,Request id,Pickup point,Driver id,Status,Request timestamp,Drop timestamp,Trip duration
0,619,Airport,1.0,Trip Completed,2016-11-07 11:51:00,2016-11-07 13:00:00,69.000000
1,867,Airport,1.0,Trip Completed,2016-11-07 17:57:00,2016-11-07 18:47:00,50.000000
2,1807,City,1.0,Trip Completed,2016-12-07 09:17:00,2016-12-07 09:58:00,41.000000
3,2532,Airport,1.0,Trip Completed,2016-12-07 21:08:00,2016-12-07 22:03:00,55.000000
4,3112,City,1.0,Trip Completed,2016-07-13 08:33:16,2016-07-13 09:25:47,52.516667


In [30]:
# Add a new column 'is_completed' that indicates whether a trip is completed or not
df['completed trips'] = df['Status'].apply(lambda x: 1 if x == 'Trip Completed' else 0)
df.head()

,Request id,Pickup point,Driver id,Status,Request timestamp,Drop timestamp,Trip duration,completed trips
0,619,Airport,1.0,Trip Completed,2016-11-07 11:51:00,2016-11-07 13:00:00,69.000000,1
1,867,Airport,1.0,Trip Completed,2016-11-07 17:57:00,2016-11-07 18:47:00,50.000000,1
2,1807,City,1.0,Trip Completed,2016-12-07 09:17:00,2016-12-07 09:58:00,41.000000,1
3,2532,Airport,1.0,Trip Completed,2016-12-07 21:08:00,2016-12-07 22:03:00,55.000000,1
4,3112,City,1.0,Trip Completed,2016-07-13 08:33:16,2016-07-13 09:25:47,52.516667,1


In [34]:
# Group the data by date and calculate the number of completed trips and the mean of trip duration on each date
completed_trips_by_date = df.loc[df['completed trips'] == 1].groupby('Request timestamp').agg({'completed trips': 'sum', 'Trip duration': 'mean'})


In [35]:
completed_trips_by_date

,completed trips,Trip duration
Request timestamp,,
2016-07-13 00:06:34,1,35.783333
2016-07-13 00:07:58,1,77.300000
2016-07-13 00:34:03,1,75.083333
2016-07-13 00:42:20,1,58.566667
2016-07-13 00:44:02,1,57.183333
...,...,...
2016-12-07 23:35:00,2,-213066.191667
2016-12-07 23:40:00,1,-213077.683333
2016-12-07 23:43:00,1,-213067.800000


In [36]:
# Group by Status and Pickup point and count the occurrences
status_pickup_counts = df[df['Status'].isin(['Cancelled', 'No Cars Available' , 'Trip Completed'])].groupby(['Status', 'Pickup point']).size()

# Display the counts
print(status_pickup_counts)


Status             Pickup point
Cancelled          Airport          198
                   City            1066
No Cars Available  Airport         1713
                   City             937
Trip Completed     Airport         1327
                   City            1504
dtype: int64


## it seems there are few cars near the airport so we need to increase No. of cars near the airport 

In [37]:
# let' Extract day of the week from 'Request timestamp'
df['Day of Week'] = df['Request timestamp'].dt.day_name()

# Count the number of requests for each day of the week
day_counts = df['Day of Week'].value_counts()

In [38]:
day_counts

Wednesday    2644
Friday       1381
Monday       1367
Thursday     1353
Name: Day of Week, dtype: int64

## we need more data for the missing days 

In [63]:
# Group by 'Day of Week', 'Status', and 'Pickup point', and count the occurrences
status_counts_per_day = df.groupby(['Pickup point' , 'Day of Week' , 'Status']).size().unstack()

In [64]:
status_counts_per_day

Status                    Cancelled  No Cars Available  Trip Completed
Pickup point Day of Week                                              
Airport      Friday              31                330             268
             Monday              42                338             281
             Thursday            32                378             191
             Wednesday           93                667             587
City         Friday             209                250             293
             Monday             220                166             320
             Thursday           220                193             339
             Wednesday          417                328             552

In [65]:
# Group by 'Driver id' and count the number of trips for each driver
driver_trip_counts = df.groupby('Driver id').size()

# Find the driver with the maximum number of trips
most_trips_driver = driver_trip_counts.idxmax()
most_trips_count = driver_trip_counts.max()

print(f"The driver with the most trips is {most_trips_driver} with {most_trips_count} trips.")


The driver with the most trips is 27.0 with 22 trips.


In [68]:
# Group by 'Driver id' and 'Day', then count the number of trips for each driver on each day
daily_trip_counts = df.groupby(['Driver id', 'Day of Week']).size()

# Find the driver with the maximum number of trips for each day
most_trips_per_day = daily_trip_counts.groupby('Day of Week').idxmax()
most_trips_count_per_day = daily_trip_counts.groupby('Day of Week').max()

# Display the results
print("Driver with the most completed trips on each day:")
for day, driver_id in most_trips_per_day.items():
    print(f"On {day}, driver {driver_id} completed the most trips with {most_trips_count_per_day[day]} trips.")


Driver with the most completed trips on each day:
On Friday, driver (70.0, 'Friday') completed the most trips with 8 trips.
On Monday, driver (22.0, 'Monday') completed the most trips with 8 trips.
On Thursday, driver (37.0, 'Thursday') completed the most trips with 7 trips.
On Wednesday, driver (54.0, 'Wednesday') completed the most trips with 12 trips.


In [74]:
driver_trips_24hrs = df.groupby(['Driver id', pd.Grouper(key='Request timestamp', freq='24H')]).size().reset_index(name='Number of Trips')

# Find the top three drivers who made the most trips within any 24-hour period
top_drivers_24hrs = driver_trips_24hrs.sort_values(by='Number of Trips', ascending=False).head(3)

# Display the result
print("Top three drivers with the most trips within any 24-hour period:")
for index, row in top_drivers_24hrs.iterrows():
    print(f"Driver {row['Driver id']} made {row['Number of Trips']} trips in a 24-hour period.")

Top three drivers with the most trips within any 24-hour period:
Driver 70.0 made 8 trips in a 24-hour period.
Driver 284.0 made 8 trips in a 24-hour period.
Driver 194.0 made 8 trips in a 24-hour period.


In [75]:
# Extract the hour of the day from the 'Request timestamp'
df['Hour'] = df['Request timestamp'].dt.hour

# Group by the hour of the day and count the number of requests for each hour
hourly_request_count = df.groupby('Hour').size()

In [77]:
hourly_request_count.sort_values(ascending = False)

Hour
18    510
20    492
19    473
21    449
5     445
9     431
8     423
17    418
7     406
6     398
22    304
10    243
4     203
23    194
12    184
15    171
11    171
13    160
16    159
14    136
2      99
0      99
3      92
1      85
dtype: int64

In [87]:
# Filter for rows where status is "No Cars Available"
no_cars_available_df = df[df['Status'] == 'No Cars Available']

# Extract the hour of the day from the 'Request timestamp'
no_cars_available_df.loc[:, 'Hour'] = df['Request timestamp'].dt.hour

# Count the number of "No Cars Available" for each hour
hourly_no_cars_available_count = no_cars_available_df.groupby('Hour').size()


C:\Users\HP\AppData\Local\Temp\ipykernel_9596\414950479.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_cars_available_df.loc[:, 'Hour'] = df['Request timestamp'].dt.hour


In [88]:
hourly_no_cars_available_count[hourly_no_cars_available_count > 90].sort_values(ascending = False)

Hour
18    322
20    290
19    283
21    265
17    232
22    138
dtype: int64

## they have to increase No. of cars at these hours ( from 17 to 22 )

## Thanks👍